<a href="https://colab.research.google.com/github/Kushagra2000/Neural_machine_translation/blob/main/Machine_translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#imports
%tensorflow_version 1.x
import tensorflow as tf
print(tf.__version__)
from tensorflow import keras
import pandas as pd
import numpy as np
import re
import unicodedata
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import LSTM, Dense, TimeDistributed, Activation,InputLayer,Embedding
from tensorflow.keras.losses import sparse_categorical_crossentropy
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
tf.test.gpu_device_name()

TensorFlow 1.x selected.
1.15.2


'/device:GPU:0'

In [2]:
#Upload the deu.txt file and put it in the Data Folder
#!ls Data/

In [3]:
#reading lines from the txt file
with open('drive/MyDrive/deu.txt','r',encoding='utf-8') as f:
  lines=(f.readlines())


In [46]:
#Hyperparameters go here
num_examples=15000
test_size=0.3
m1_lr=0.001
m2_lr=0.005
m1_lstm_units=64
m2_lstm_units=64
m1_epochs=25
m2_epochs=30
m1_batch_size=128
m2_batch_size=128
m2_embedding_col=64

In [5]:
#removing the unnecessary data and obtaining sentence pairs 
def process_text(lines,translate_mode):
  ''' 
  Takes a list of tab seperated
  English and German sentences as input 
  and returns dataframe of processed
  English and German sentence pairs
  '''
  proc_lines=[]
  if translate_mode==False:
    for line in lines:
      line=line.strip() #to remove newlines at the end of a sentence
      line=line.split("\t") #splitting by tabs
      line=line[:-1]  #remove contributing information
      
      line[0]=''.join(c for c in unicodedata.normalize('NFD', line[0]) if unicodedata.category(c) != 'Mn')
      line[0]=line[0].encode('utf8','ignore').decode('utf8')
      line[0]=line[0].replace(u'\u200b',' ')
      line[0]=line[0].lower()
      line[0]=line[0].replace('\xa0', ' ')
      line[0]=re.sub(r"([.,?!;])",r" \1 ",line[0])   #adding spaces before and after punctuation
      line[0]=re.sub(r"[0-9]"," ",line[0])
      line[0]=re.sub(r'["]'," ",line[0])
      line[0]=re.sub(r"[']","",line[0])
      line[0]=re.sub(r"[%-,]"," ",line[0])
      line[0]=re.sub(r"[:]"," ",line[0])
      line[0]=re.sub(r'[" "]+'," ",line[0])  #removing excess spaces
      line[0]=line[0].strip() #removing spaces from the end of string
      line[0]="<SOS> "+line[0]+" <EOS>"

      line[1]=''.join(c for c in unicodedata.normalize('NFD', line[1]) if unicodedata.category(c) != 'Mn')
      line[1]=line[1].replace(u'\u200b',' ')
      line[1]=line[1].replace('\xa0', ' ')
      line[1]=line[1].lower()
      line[1]=re.sub(r"([.,?!;])",r" \1 ",line[1])
      line[1]=re.sub(r"[0-9]",r" ",line[1])
      line[1]=re.sub(r'["]',"",line[1])
      line[1]=re.sub(r'[—]',"",line[1])
      line[1]=re.sub(r'[„]',"",line[1])
      line[1]=re.sub(r'[“]',"",line[1])
      line[1]=re.sub(r'[–]',"",line[1])
      line[1]=re.sub(r'[‘‚]',"",line[1])
      line[1]=re.sub(r"[']","",line[1])
      line[1]=re.sub(r"[%()-,]"," ",line[1])
      line[1]=re.sub(r"[:]"," ",line[1])
      line[1]=re.sub(r'[" "]+'," ",line[1])
      line[1]=line[1].strip()
      line[1]="<SOS> "+line[1]+" <EOS>"

      proc_lines.append(line) 
    return pd.DataFrame.from_records(data=proc_lines,columns=['eng','ger'])
  else:
    lines=lines.strip()
    lines=''.join(c for c in unicodedata.normalize('NFD', lines) if unicodedata.category(c) != 'Mn')
    lines=lines.encode('utf8','ignore').decode('utf8')
    lines=lines.replace(u'\u200b',' ')
    lines=lines.lower()
    lines=lines.replace('\xa0', ' ')
    lines=re.sub(r"([.,?!;])",r" \1 ",lines)   #adding spaces before and after punctuation
    lines=re.sub(r"[0-9]"," ",lines)
    lines=re.sub(r'["]'," ",lines)
    lines=re.sub(r"[']","",lines)
    lines=re.sub(r"[%-,]"," ",lines)
    lines=re.sub(r"[:]"," ",lines)
    lines=re.sub(r'[" "]+'," ",lines)  #removing excess spaces
    lines=lines.strip() #removing spaces from the end of string
    lines="<SOS> "+lines+" <EOS>"
    return lines

  
     
  

In [6]:
#reading in the dataset, sampling randomly, shuffling and removing duplicates 
df=process_text(lines,False)
df=df.sample(n=num_examples, random_state=1)
df.drop_duplicates(subset="eng",inplace=True)
df = df.sample(frac=1).reset_index(drop=True)
df.head(10)

,eng,ger
0,<SOS> doesnt tom go to boston every month ? <EOS>,<SOS> fahrt tom nicht jeden monat nach boston ...
1,<SOS> the doctor told tom to stay in bed and g...,<SOS> der arzt wies tom an bettruhe einzuhalte...
2,<SOS> whats your favorite hot drink ? <EOS>,<SOS> was ist dein liebstes warmes getrank ? <...
3,<SOS> he died of lung cancer . <EOS>,<SOS> er starb an lungenkrebs . <EOS>
4,<SOS> hes always scowling . <EOS>,<SOS> er schaut immer bose drein . <EOS>
5,<SOS> it would be fun to see how things change...,<SOS> es ware lustig zu sehen wie die dinge si...
6,<SOS> will you lend me your pencil ? <EOS>,<SOS> leihst du mir deinen stift ? <EOS>
7,<SOS> tom sat at the bar with mary . <EOS>,<SOS> tom saß mit mary in der bar . <EOS>
8,<SOS> i want you to go to osaka at once . <EOS>,<SOS> ich will dass sie sofort nach osaka fahr...
9,<SOS> will you let tom leave ? <EOS>,<SOS> lasst ihr tom gehen ? <EOS>


In [7]:
df.tail(10)

,eng,ger
14706,<SOS> tom and mary quarreled . <EOS>,<SOS> tom und maria stritten sich . <EOS>
14707,<SOS> this doesnt taste like pork to me . <EOS>,<SOS> fur mich schmeckt das nicht nach schwein...
14708,<SOS> tom has two brothers . one lives in bost...,<SOS> tom hat zwei bruder . der eine lebt in b...
14709,<SOS> the united states is the largest produce...,<SOS> die vereinigten staaten sind der großte ...
14710,<SOS> she can sew very well . <EOS>,<SOS> sie kann ziemlich gut nahen . <EOS>
14711,<SOS> i have a friend whos a vegetarian . <EOS>,<SOS> ich habe einen freund der vegetarier ist...
14712,<SOS> this is a defibrillator . <EOS>,<SOS> das ist ein defibrillator . <EOS>
14713,<SOS> were sad . <EOS>,<SOS> wir sind traurig . <EOS>
14714,<SOS> i think i remember you . <EOS>,<SOS> ich glaube ich erinnere mich an sie . <EOS>
14715,<SOS> eggs are very fragile . <EOS>,<SOS> eier sind sehr zerbrechlich . <EOS>


In [8]:
class Vocab_builder():
  '''
  Builds vocabulary and 
  word to index and index to word dictionaries
  from dataset
  '''
  def __init__(self,lang,series):
    self.lang=lang
    self.data=series
  def tokenize(self,line):
    return line.split(' ')
  def build_vocab(self):
    self.uniq_words=set()
    
    self.maxlen=0
    count=3
    self.num_list=[]
    for index,line in self.data.items():
      self.word_list=self.tokenize(line)
      self.maxlen=max(len(self.word_list),self.maxlen)
      for word in self.word_list:
        if(word not in self.uniq_words and word!='<EOS>' and word!='<SOS>'):
          self.uniq_words.add(word)
          self.num_list.append(count)
          count+=1
      
    self.vocab_list=['<PAD>','<SOS>','<EOS>']+sorted(list(self.uniq_words))
    self.num_list=[0,1,2]+self.num_list
    print("Built vocabulary having {} elements".format(len(self.vocab_list)))
    print("Largest sentence length (with tags):{}".format(self.maxlen))
    return dict(zip(self.vocab_list,self.num_list)),dict(zip(self.num_list,self.vocab_list))






In [9]:
#Objects of Vocab_builder class
eng=Vocab_builder('eng',df['eng'])
ger=Vocab_builder('ger',df['ger'])


In [10]:
#English word to index and index to word dictionaries
eng_w2i,eng_i2w=eng.build_vocab()


Built vocabulary having 5903 elements
Largest sentence length (with tags):31


In [11]:
#checking for special characters in English dictionary
for i in sorted (eng_w2i.keys())[:20] :  
     print(i, end = " ") 

! $ -to- . ; <EOS> <PAD> <SOS> ? a abandon abandoned abducted ability able aboard about above abroad absent 

In [12]:
#German word to index and index to word dictionaries
ger_w2i,ger_i2w=ger.build_vocab()

Built vocabulary having 9004 elements
Largest sentence length (with tags):30


In [15]:
#checking if the index dictionaries are correct
'geh'==ger_i2w[ger_w2i['geh']]

True

In [16]:
'go'==eng_i2w[eng_w2i['go']]

True

In [17]:
#checking for special characters in German dictionary
for i in sorted (ger_w2i.keys())[:20] :  
     print(i, end = " ") 

! $ - . ; <EOS> <PAD> <SOS> ? ab abbekommen abbezahlen abbiegen abbringen abend abendbrot abende abendessen abendkleid abendkleider 

In [63]:
def sent_to_ind(sentence,lang):
  '''
  Tokenizes a string and
  converts it to an np array of 
  indices and pads the 
  array according to max sentence length
  '''
  ind_list=[]
  if lang=='eng':
    tokens=eng.tokenize(sentence)
    for token in tokens:
      ind_list.append(eng_w2i[token])
    while len(ind_list)<max(ger.maxlen,eng.maxlen):
      ind_list.append(0)
  else:
    tokens=ger.tokenize(sentence)
    for token in tokens:
      ind_list.append(ger_w2i[token])
    while len(ind_list)<max(ger.maxlen,eng.maxlen):
      ind_list.append(0)
    
  return np.array(ind_list)



In [22]:
#Checking correctness of sentence to index conversion
test_lis=sent_to_ind('<SOS> tom wird bald hier sein . was heißt bald ? <EOS>','ger')

print(len(test_lis))
print(test_lis)

31
[   1 7289 8594  760 3778 6624    6 8352 3693  760    8    2    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0]


In [23]:
test_lis=sent_to_ind("<SOS> tom will be here soon . how soon ? <EOS>",'eng')
print(len(test_lis))
print(test_lis)

31
[   1 5312 5770  457 2477 4788    6 2570 4788    8    2    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0]


In [29]:
#splitting the data into training,testing and validation sets

train_x,test_x,train_y,test_y=train_test_split(df['eng'],df['ger'],test_size=0.1,random_state=42)
train_x,val_x,train_y,val_y=train_test_split(train_x,train_y,test_size=0.23,random_state=42)


In [30]:
def sent_to_np(series,lang,translate_mode):
  '''
  Converts a dataframe column to 
  a unsqueezed np array of indexes
  with padding for feeding into NN
  '''
  ret_list=[]
  if translate_mode==False :
    if lang=='eng':
      for index,val in series.items():
        ret_list.append(sent_to_ind(val,'eng'))
    else:
      for index,val in series.items():
        ret_list.append(sent_to_ind(val,'ger'))
    
    ret_list=np.array(ret_list)
    return np.expand_dims(ret_list,axis=2)
  else:
    ans=sent_to_ind(series,'eng')
    ans=np.expand_dims(ans,axis=0)
    ans=np.expand_dims(ans,axis=2)
    return ans


In [31]:
train_x=sent_to_np(train_x,'eng',False)
train_y=sent_to_np(train_y,'ger',False)
test_x=sent_to_np(test_x,'eng',False)
test_y=sent_to_np(test_y,'ger',False)
val_x=sent_to_np(val_x,'eng',False)
val_y=sent_to_np(val_y,'ger',False)


In [ ]:
#loading training,testing and validation data

# train_x=np.load("drive/MyDrive/data/training/train_x.npy")
# train_y=np.load("drive/MyDrive/data/training/train_y.npy")

# test_x=np.load("drive/MyDrive/data/testing/test_x.npy")
# test_y=np.load("drive/MyDrive/data/testing/test_y.npy")

# val_x=np.load("drive/MyDrive/data/validation/val_x.npy")
# val_y=np.load("drive/MyDrive/data/validation/val_y.npy")



In [32]:
print(train_x.shape)
print(train_y.shape)

(10197, 31, 1)
(10197, 31, 1)


In [33]:
print(test_x.shape)
print(test_y.shape)

(1472, 31, 1)
(1472, 31, 1)


In [34]:
print(val_x.shape)
print(val_y.shape)

(3047, 31, 1)
(3047, 31, 1)


In [35]:
def translate(sentence,mod,embedded):
  '''
  Function for translating given English sentence
  to German using model predictions
  '''
  ans=[]
  preproc_sent=process_text(sentence,True)
  #print(preproc_sent)
  model_inp=sent_to_np(preproc_sent,'eng',True)
  if (embedded):
    model_inp=np.squeeze(model_inp,axis=2)
  #print(model_inp)
  pred=mod.predict(model_inp)
  #print(pred)
  for i in pred[0]:
    ind=np.argmax(i)
    #print(ind)
    #print("MAX:",i[ind])
    #print(i[24])
    ans.append(ger_i2w[ind])
  return str(ans)

In [36]:
#training starts here ----------------------------------------------------------

In [37]:
# def f1_score(y_true,y_pred):
#   p=tf.keras.metrics.Precision()
#   r=tf.keras.metrics.Recall()
#   p.update_state(y_true,y_pred)
#   r.update_state(y_true,y_pred)
#   return (2*p.result()*r.result())/(p.result()+r.result());


def base_LSTM_model(m1_lr,m1_lstm_units):
  '''
  Simple LSTM model
  '''
  lstm=LSTM(m1_lstm_units,return_sequences=True,activation='tanh')  #LSTM layer with output being hiddent state at time t
  layer_at_t=TimeDistributed(Dense(len(ger.vocab_list),activation='softmax')) #Dense layer acting on hidden output at each step to generate predictions
  model=tf.keras.Sequential()
  model.add(InputLayer(train_x.shape[1:]))
  model.add(lstm)
  model.add(layer_at_t)

  model.compile(loss=sparse_categorical_crossentropy,optimizer=Adam(m1_lr),metrics=['accuracy','MeanSquaredError'])
  return model


In [38]:
base_model=base_LSTM_model(m1_lr,m1_lstm_units)
base_model.summary()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [39]:
#training the base model

base_model.fit(train_x,train_y,m1_batch_size,m1_epochs,validation_data=(val_x,val_y))
#base_model.save("Less_trained_base.h5")

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 10197 samples, validate on 3047 samples
Epoch 1/25
10197/10197 [==============================] - 9s 898us/sample - loss: 5.6665 - acc: 0.6701 - mean_squared_error: 5503041.5000 - val_loss: 2.2605 - val_acc: 0.6959 - val_mean_squared_error: 5536401.0000
Epoch 2/25
10197/10197 [==============================] - 7s 693us/sample - loss: 2.0554 - acc: 0.7004 - mean_squared_error: 5503041.5000 - val_loss: 1.9712 - val_acc: 0.7031 - val_mean_squared_error: 5536400.5000
Epoch 3/25
10197/10197 [==============================] - 7s 666us/sample - loss: 1.9074 - acc: 0.7252 - mean_squared_error: 5503041.0000 - val_loss: 1.8987 - val_acc: 0.7372 - val_mean_squared_error: 5536401.0000
Epoch 4/25
10197/10197 [==============================] - 7s 671us/sample - loss: 1.8552 - acc: 0.7431 - mean_squared_error: 5503041.5000 - val_loss: 1.8662 - val_acc: 0.7402 - val_mean_squared_error: 5536400.5000
E

In [ ]:
# base_model=tf.keras.models.load_model("drive/MyDrive/model/Less_trained_base.h5")
# base_model.summary()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 113, 64)           16896     
_________________________________________________________________
time_distributed (TimeDistri (None, 113, 34809)        2262585   
Total params: 2,279,481
Trainable params: 2,279,481
Non-trainable params: 0
_____________________________________________

In [42]:
#Evaluating LSTM model on unseen data

l,acc,mse=base_model.evaluate(test_x,test_y)
print("Base model loss for testing set:{}".format(l))
print("Base model accuracy for testing set:{}".format(acc))
print("Base model MSE for testing set:{}".format(mse))


1472/1472 [==============================] - 1s 780us/sample - loss: 1.5307 - acc: 0.7672 - mean_squared_error: 5426667.5000
Base model loss for testing set:1.5306736075359841
Base model accuracy for testing set:0.7672028541564941
Base model MSE for testing set:5426667.5


In [44]:
#Sample translation by base LSTM model
translate("I am sad",base_model,False)

"['<SOS>', 'ich', 'ist', 'nicht', '.', '<EOS>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>']"

In [45]:
#embedding model starts here-------------------------------------------

#Removing singleton axis from 3rd axis
#for embedding layer
t_x=np.squeeze(train_x,axis=2)
v_x=np.squeeze(val_x,axis=2)
te_x=np.squeeze(test_x,axis=2)
print(t_x.shape)
print(v_x.shape)
print(te_x.shape)

(10197, 31)
(3047, 31)
(1472, 31)


In [49]:

def embedding_LSTM(m2_lstm_units,lr,embedding_col):
  '''
  LSTM model with embedding layer
  '''
  lstm=LSTM(m2_lstm_units,return_sequences=True,activation="tanh")
  print(t_x.shape[1])
  embed=Embedding(len(ger.vocab_list),embedding_col,input_length=t_x.shape[1])
  layer_at_t=TimeDistributed(Dense(len(ger.vocab_list),activation="softmax"))
  model=tf.keras.Sequential()
  model.add(embed)
  model.add(lstm)
  model.add(layer_at_t)

  model.compile(loss=sparse_categorical_crossentropy,optimizer=Adam(lr),metrics=['accuracy','MeanSquaredError'])
  return model
  

In [50]:
embeded_model=embedding_LSTM(m2_lstm_units,m2_lr,m2_embedding_col)

31


In [51]:
#training embedding model
embeded_model.fit(t_x,train_y,batch_size=m2_batch_size,epochs=m2_epochs,validation_data=(v_x,val_y))

Train on 10197 samples, validate on 3047 samples
Epoch 1/30
10197/10197 [==============================] - 8s 741us/sample - loss: 3.2681 - acc: 0.6973 - mean_squared_error: 5503041.0000 - val_loss: 1.9890 - val_acc: 0.7282 - val_mean_squared_error: 5536400.5000
Epoch 2/30
10197/10197 [==============================] - 7s 689us/sample - loss: 1.7437 - acc: 0.7304 - mean_squared_error: 5503043.0000 - val_loss: 1.6639 - val_acc: 0.7329 - val_mean_squared_error: 5536401.0000
Epoch 3/30
10197/10197 [==============================] - 7s 700us/sample - loss: 1.5771 - acc: 0.7449 - mean_squared_error: 5503040.0000 - val_loss: 1.5940 - val_acc: 0.7508 - val_mean_squared_error: 5536401.0000
Epoch 4/30
10197/10197 [==============================] - 7s 706us/sample - loss: 1.5154 - acc: 0.7573 - mean_squared_error: 5503041.0000 - val_loss: 1.5517 - val_acc: 0.7554 - val_mean_squared_error: 5536401.0000
Epoch 5/30
10197/10197 [==============================] - 7s 685us/sample - loss: 1.4696 - acc:

In [59]:
embeded_model=tf.keras.models.load_model("drive/MyDrive/embedded_15000.h5")
l,acc=embeded_model.evaluate(te_x,test_y)
print("Embedded model loss for testing set:{}".format(l))
print("Embedded model accuracy for testing set:{}".format(acc))
#print("Embedded model MSE for testing set:{}")

1472/1472 [==============================] - 1s 794us/sample - loss: 0.7717 - acc: 0.8711
Embedded model loss for testing set:0.7716904360315074
Embedded model accuracy for testing set:0.871077299118042


In [61]:
translate("I am sad",embeded_model,True)

"['<SOS>', 'ich', 'bin', 'traurig', '.', '<EOS>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>']"

In [ ]:
#Hyperparameter tuning starts here
